# Лабораторная работа №3 (проведение исследований с алгоритмом DecisionTree)

In [137]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, max_error



In [138]:
car_dataset = fetch_ucirepo(id=9)

In [139]:
df = pd.DataFrame(data=car_dataset.data.features)
df['mpg'] = car_dataset.data.targets

In [140]:
target_column = "mpg"

In [141]:
df["horsepower"] = df["horsepower"].fillna(df["horsepower"].median())

Разобьем выборку на обучающую и валидационную

In [142]:
X = df.drop(columns=target_column, inplace=False)
y = df[target_column]

In [143]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. Создание бейзлайна и оценка качества

### Обучение модели

Определим модель и обучим модель

In [144]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

DecisionTreeRegressor()

Сделаем предсказания

In [145]:
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)

### Оценка качества модели

In [146]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(y_train, train_predict)
train_mape = mean_absolute_percentage_error(y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(y_train, train_predict)
train_max_error = max_error(y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(y_valid, valid_predict)
valid_max_error = max_error(y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [147]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,0.0,2.220000
1,MAPE (%),0.0,9.649104
2,R²,1.0,0.791092
3,Max Error,0.0,16.000000


## 3. Улучшение бейзлайна

### Препроцессинг данных

In [148]:
df_optimized = df.copy()

In [149]:
# Обработка выбросов
def remove_outliers(data, columns, threshold=1.5):
    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound)]
    return data

df_optimized = remove_outliers(df_optimized, df_optimized.columns)

In [150]:
X = df_optimized.drop(target_column, axis=1)
y = df_optimized[target_column]

Скалирование данных

In [151]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [152]:
new_X_train, new_X_valid, new_y_train, new_y_valid = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Обучение модели с новыми данными

Переопределим модель

In [155]:
model = DecisionTreeRegressor()

In [156]:
model.fit(new_X_train, new_y_train)

DecisionTreeRegressor()

Сделаем предсказания

In [157]:
train_predict = model.predict(new_X_train)
valid_predict = model.predict(new_X_valid)

### Оценка качетсва модели с улучшеным бейзлайном

In [158]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(new_y_train, train_predict)
train_mape = mean_absolute_percentage_error(new_y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(new_y_train, train_predict)
train_max_error = max_error(new_y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(new_y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(new_y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(new_y_valid, valid_predict)
valid_max_error = max_error(new_y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [159]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,0.0,2.243421
1,MAPE (%),0.0,9.659978
2,R²,1.0,0.785841
3,Max Error,0.0,14.800000


### Резюме:
На качество модели повлияли, как входные данные, так и параметры самой модели:
- При помощи квантилей я удалил выбросы
- Скалировал данные StandartScaller'ом
- Избавился от высокой кореляции фичей

## 4. Имплементация алгоритма машинного обучения

### Реализация модели

In [160]:
class Node:
    def __init__(self, threshold=None, idx_feature=None, criterion=None, parent=None, left_children=None, right_children=None, y_values=None):
        self.threshold = threshold
        self.idx_feature = idx_feature
        self.parent = parent
        self.left_children = left_children
        self.right_children = right_children
        self.y_values = y_values
        self.criterion = criterion

class DecisionTree:
    def __init__(self, max_depth=1000, min_leaf_split=2):
        self.root = None
        self.max_depth = max_depth
        self.min_leaf_split = min_leaf_split

    def fit(self, X, y):
        X, y = self._prepare_inputs(X, y)
        self.root = self.build_tree(X, y)

    def build_tree(self, X, y, parent=None, depth=0):
        if X.shape[0] <= self.min_leaf_split or depth >= self.max_depth:
            return Node(parent=parent, y_values=y)

        best_criterion, best_idx_feature, best_threshold = self.find_best_feature_split(X, y)

        if best_criterion is None or best_criterion <= 0:
            return Node(parent=parent, y_values=y)

        node = Node(threshold=best_threshold, idx_feature=best_idx_feature, criterion=best_criterion, parent=parent, y_values=y)

        left_mask = X[:, best_idx_feature] <= best_threshold
        right_mask = ~left_mask
        left_X, left_y = X[left_mask], y[left_mask]
        right_X, right_y = X[right_mask], y[right_mask]

        node.left_children = self.build_tree(left_X, left_y, parent=node, depth=depth + 1)
        node.right_children = self.build_tree(right_X, right_y, parent=node, depth=depth + 1)

        return node

    def predict(self, X) -> np.ndarray:
        X = self._prepare_inputs(X)
        def traverse_batch(node, X_batch):
            if node.left_children is None and node.right_children is None:
                return np.full(X_batch.shape[0], self._predict_leaf(node), dtype=float)

            left_mask = X_batch[:, node.idx_feature] <= node.threshold
            right_mask = ~left_mask

            predictions = np.empty(X_batch.shape[0], dtype=float)
            if left_mask.any():
                predictions[left_mask] = traverse_batch(node.left_children, X_batch[left_mask])
            if right_mask.any():
                predictions[right_mask] = traverse_batch(node.right_children, X_batch[right_mask])
            return predictions

        return traverse_batch(self.root, X)



    def find_best_feature_split(self, X: np.ndarray, y: np.ndarray):
        best_idx_feature = None
        best_threshold = None
        best_criterion = 0

        for idx_feature in range(X.shape[1]):
            cur_criterion, cur_threshold = self.find_best_threshold_split(X[:, idx_feature], y)
            if cur_criterion > best_criterion:
                best_threshold = cur_threshold
                best_criterion = cur_criterion
                best_idx_feature = idx_feature

        return best_criterion, best_idx_feature, best_threshold

    def find_best_threshold_split(self, X_cols: np.ndarray, y: np.ndarray):
        best_criterion = 0
        best_threshold = None

        uniq_elem = np.unique(X_cols)
        for i in range(1, len(uniq_elem)):
            threshold = (uniq_elem[i-1] + uniq_elem[i]) / 2
            y_left, y_right = self.get_split(X_cols, y, threshold)
            cur_criterion = self.calculate_criterion(y, y_left, y_right)
            if cur_criterion > best_criterion:
                best_criterion = cur_criterion
                best_threshold = threshold

        return best_criterion, best_threshold

    def calculate_criterion(self, y: np.ndarray, y_left: np.ndarray, y_right: np.ndarray):
        total_len = len(y)
        len_left = len(y_left)
        len_right = len(y_right)

        if len_left == 0 or len_right == 0:
            return 0.0

        return (
            self.criterion(y)
            - (len_left / total_len * self.criterion(y_left)
            + len_right / total_len * self.criterion(y_right))
        )

    @staticmethod
    def get_split(X_cols, y, threshold):
        indices = X_cols <= threshold
        left = y[indices]
        right = y[~indices]
        return left, right

    @staticmethod
    def _prepare_inputs(X, y=None):
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            X = X.values
        if isinstance(X, list):
            X = np.array(X)
        X = X.astype(float)

        if y is not None:
            if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
                y = y.values
            if isinstance(y, list):
                y = np.array(y)
            if isinstance(y, pd.CategoricalDtype):
                y = y.astype(int)
            y = y.astype(float)
            return X, y

        return X

class MyDecisionTreeRegressor(DecisionTree):
    def __init__(self, max_depth=10, min_leaf_split=2):
        super().__init__(max_depth, min_leaf_split)
        self.criterion = self.calculate_mse

    def _predict_leaf(self, node: Node):
        if node.y_values.size == 0:
            return 0.0
        return node.y_values.mean()

    @staticmethod
    def calculate_mse(y: np.ndarray):
        if y.size == 0:
            return 0.0
        y_pred = np.mean(y)
        mse = np.mean((y - y_pred) ** 2)
        return mse

### Обучение 

In [161]:
model = MyDecisionTreeRegressor()
model.fit(X_train, y_train)

In [162]:
train_predict = model.predict(X_train)
valid_predict = model.predict(X_valid)

###  Оценка качества модели

In [163]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(y_train, train_predict)
train_mape = mean_absolute_percentage_error(y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(y_train, train_predict)
train_max_error = max_error(y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(y_valid, valid_predict)
valid_max_error = max_error(y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [164]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,0.318913,2.214435
1,MAPE (%),1.399476,9.635800
2,R²,0.993559,0.789462
3,Max Error,4.166667,16.000000


### Обучение с улучшенным бейзлайном

In [165]:
model = Pipeline((
    ("pca", PCA(n_components=4)),
    ("model", MyDecisionTreeRegressor())
))

In [166]:
model.fit(new_X_train, new_y_train)

Pipeline(steps=[('pca', PCA(n_components=4)),
                ('model',
                 <__main__.MyDecisionTreeRegressor object at 0x0000026258D1CB20>)])

In [167]:
train_predict = model.predict(new_X_train)
valid_predict = model.predict(new_X_valid)

###  Оценка качества модели

In [168]:
# Расчет метрик для тренировочных предсказаний
train_mae = mean_absolute_error(new_y_train, train_predict)
train_mape = mean_absolute_percentage_error(new_y_train, train_predict) * 100  # Преобразуем в проценты
train_r2 = r2_score(new_y_train, train_predict)
train_max_error = max_error(new_y_train, train_predict)

# Расчет метрик для валидационных предсказаний
valid_mae = mean_absolute_error(new_y_valid, valid_predict)
valid_mape = mean_absolute_percentage_error(new_y_valid, valid_predict) * 100  # Преобразуем в проценты
valid_r2 = r2_score(new_y_valid, valid_predict)
valid_max_error = max_error(new_y_valid, valid_predict)

# Сводка метрик в таблицу для удобного отображения
metrics_df = pd.DataFrame({
    'Метрика': ['MAE', 'MAPE (%)', 'R²', 'Max Error'],
    'Тренировочная выборка': [train_mae, train_mape, train_r2, train_max_error],
    'Валидационная выборка': [valid_mae, valid_mape, valid_r2, valid_max_error]
})

In [169]:
metrics_df.head()

,Метрика,Тренировочная выборка,Валидационная выборка
0,MAE,0.396516,2.470865
1,MAPE (%),1.622162,10.259909
2,R²,0.989564,0.769749
3,Max Error,5.700000,16.600000
